In [1]:
%matplotlib qt5

from pathlib import Path
import pickle

from orix import io, plot
# from orix.quaternion import Orientation
from orix.vector import Vector3d

import hyperspy.api as hs #General hyperspy package
import pyxem as pxm #Electron diffraction tools based on hyperspy
import numpy as np #General numerical and matrix support
import matplotlib.pyplot as plt #Plotting tools
import matplotlib.colors as mcolors #Some plotting color tools
from matplotlib.cm import ScalarMappable
import diffpy #Electron diffraction tools
import requests

# import pandas as pd
# import seaborn as sb

#Import indexation and plotting tools
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from pyxem.utils import expt_utils as eutls

#Import orientation handling tools
from orix.quaternion import Rotation, symmetry, Orientation
from orix.vector.vector3d import Vector3d
from orix.projections import StereographicProjection
from orix import plot

import matplotlib
matplotlib.rcParams["backend"] = "Agg"
from orix.crystal_map.crystal_map import CrystalMap

## One notebook for all the processes needed to template match centered data

## Template matching

In [ ]:
def change_step_size(xmap, pixel_size, scan_unit):
    """Change the step size of an orix CrystalMap
    
    """
    x = xmap.x * pixel_size
    y = xmap.y * pixel_size
    rot = xmap.rotations
    phaseid = xmap.phase_id
    prop = xmap.prop
    is_in_data = xmap.is_in_data
    phaselist = xmap.phases
    new_xmap = CrystalMap(rotations = rot,
                        phase_id = phaseid,
                        x = x,
                        y = y,
                        prop = prop,
                        scan_unit = scan_unit,
                        is_in_data = is_in_data,
                        phase_list=phaselist)
    return new_xmap

In [3]:
# after the dataset is processed
def tm_and_plot(libpath,datapath,name):
    # rot_z = 150.
    # rot_z = 180-310. # cycled
    # pixel_size = 15.356
    # scan_unit = "nm"
    # load data
    with open(libpath, 'rb') as fp: # With b sub
        library = pickle.load(fp)
    processed_data = hs.load(str(datapath), lazy=True)
    cs = symmetry.Oh
    # Correct scale
    scale = 0.009451434347767504
    processed_data.set_diffraction_calibration(scale)
    processed_data.set_scan_calibration(scale)

    frac_keep = 1 
    result, phasedict = iutls.index_dataset_with_template_rotation(processed_data.inav[::1],
                                                        library,
                                                    phases = ["LNMO"],  # if we have multiple phases we can also specify which ones we want to consider. If it's not specified, all phases are used.
                                                    n_best = 1, # CHANGE BACK TO 3
                                                    frac_keep = frac_keep,
                                                    n_keep = None,
                                                    delta_r = 1,
                                                    delta_theta = 1,
                                                    max_r = np.min(processed_data.axes_manager.signal_shape)//2,
                                                    intensity_transform_function=lambda x: x**0.09,#x**0.15,#c*np.log(1+x),#np.where(x>0.0006,x**0.15,-1.2),# x**0.15, #None,
                                                    normalize_images = True,
                                                    normalize_templates=True,
                                                    )
    
    # Rotate
    # result["orientation"][:,:,:,0] = result["orientation"][:,:,:,0] + rot_z
    # Set scale
    
    xmap = iutls.results_dict_to_crystal_map(result, phasedict, diffraction_library=library)
    xmap.phases[0].space_group = 227 
    # xmap.phases[0].space_group = 212

    xmap.correlation = np.nan_to_num(xmap.correlation) # If any correlation scores are NaN
    # xmap = change_step_size(xmap, pixel_size, scan_unit)

    # Save in both places
    io.save(f"./small_sample/{name}.h5", xmap) 
    io.save(f"./small_sample/{name}.ang",xmap)
    # io.save("./small_sample/small_xmap1.h5",xmap)

    # Make crystal map
    ipf_key = plot.IPFColorKeyTSL(xmap.phases[0].point_group, direction=Vector3d.xvector())
    rgb = ipf_key.orientation2color(xmap.orientations)
    fig = xmap.plot(rgb, remove_padding=True, return_figure=True)


In [ ]:
# tm_and_plot("./Libraries/lib_9.pkl","./Midway_data/processed_image_v6.zspy","xmap1_v8")

## Run for multiple datasets

In [ ]:
lib_path = "D:/Project_thesis/Code_data/lib_9.pkl"
# data_list = ["./Midway_data_post/processed_image_v6.zspy","./Midway_data_post/processed_image2_v6.zspy","./Midway_data_post/processed_image3_v6.zspy"]
# data_list = ["D:/Project_thesis/Code_data/Sample_data/20230616_130045_centered.zspy",
#              "D:/Project_thesis/Code_data/Sample_data/20230616_145334_centered.zspy",
#              "D:/Project_thesis/Code_data/Sample_data/20230616_153716_centered.zspy"]
# data_list = ["D:/Project_thesis/Code_data/Sample_data_post/20231010_171125_centered.zspy",
#              "D:/Project_thesis/Code_data/Sample_data_post/20231010_182535_centered.zspy",
#              "D:/Project_thesis/Code_data/Sample_data_post/20231010_192356_centered.zspy"]

data_list = [] # 20230616_130045, 20230616_145334, 20230616_153716
# data_list = [] # 20231010_171125, 20231010_182535, 20231010_192356
name_list = ["xmap1_whole","xmap2_whole","xmap3_whole"]
# name_list = ["xmap1_post_whole","xmap2_post_whole","xmap3_post_whole"]


for i in range(0,3):
    tm_and_plot(lib_path,data_list[i],name_list[i])

In [4]:
tm_and_plot("./Libraries/lib_other_structure.pkl",'./small_sample/tilt1_small.zspy','xmap1_small')

[#################                       ] | 42% Completed | 39m 42ss

c:\Users\KajaEA\miniconda3\envs\pyxem\Lib\site-packages\pyxem\utils\indexation_utils.py:877: RuntimeWarning: invalid value encountered in divide
  polar_image = polar_image / dispatcher.linalg.norm(polar_image)


[########################################] | 100% Completed | 102m 2s
